In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers.experimental.preprocessing import StringLookup
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import matplotlib.image as mpimg


In [2]:
styles = pd.read_csv("D:/Btech/college/SEM7/archive (1)/styles.csv", error_bad_lines=False)
styles["usage"].unique()    

b'Skipping line 6044: expected 10 fields, saw 11\nSkipping line 6569: expected 10 fields, saw 11\nSkipping line 7399: expected 10 fields, saw 11\nSkipping line 7939: expected 10 fields, saw 11\nSkipping line 9026: expected 10 fields, saw 11\nSkipping line 10264: expected 10 fields, saw 11\nSkipping line 10427: expected 10 fields, saw 11\nSkipping line 10905: expected 10 fields, saw 11\nSkipping line 11373: expected 10 fields, saw 11\nSkipping line 11945: expected 10 fields, saw 11\nSkipping line 14112: expected 10 fields, saw 11\nSkipping line 14532: expected 10 fields, saw 11\nSkipping line 15076: expected 10 fields, saw 12\nSkipping line 29906: expected 10 fields, saw 11\nSkipping line 31625: expected 10 fields, saw 11\nSkipping line 33020: expected 10 fields, saw 11\nSkipping line 35748: expected 10 fields, saw 11\nSkipping line 35962: expected 10 fields, saw 11\nSkipping line 37770: expected 10 fields, saw 11\nSkipping line 38105: expected 10 fields, saw 11\nSkipping line 38275: ex

array(['Casual', 'Ethnic', 'Formal', 'Sports', nan, 'Smart Casual',
       'Travel', 'Party', 'Home'], dtype=object)

In [3]:
def df_drop(styles, col, item):
    """
    This function drops certain columns
    input: styles, dataframe
        col, the item we want to drop in this coloumn
        item, which item we want to drop 
    """
    for i in item:
        styles = styles.drop(styles[styles[col] == i].index)
    return styles

In [4]:
####LOAD DATASET
def get_df():
    styles = pd.read_csv("D:/Btech/college/SEM7/archive (1)/styles.csv", error_bad_lines=False)
    styles = styles.drop(["productDisplayName"],axis = 1) #drop useless column, we do not need name to do recommendation
    styles = styles.drop(["year"],axis = 1) #drop useless column, we do not need year to do recommendation
    styles = styles[(styles.masterCategory=='Apparel')| (styles.masterCategory=='Footwear')] # drop useless rows, we are not recommend acessories
    styles = styles.drop(styles[styles["subCategory"] == "Innerwear"].index) # drop useless row, we are not recommend innerwears, only outfits.
    styles = styles.dropna() # drop NA
    styles = df_drop(styles,"subCategory", ["Apparel Set", "Dress","Loungewear and Nightwear","Saree","Socks"]) # we only recommend outfits.
    styles["subCategory"] = styles["subCategory"].transform(lambda x: "Footwear" if(x in ["Shoes","Flip Flops","Sandal"]) else x) # Group them into one category.
    styles = styles.drop(labels=[6695,16194,32309,36381,40000], axis=0) # drop incomplete rows
    return styles

styles = get_df()
#styles.head()
styles["subCategory"].unique() # we can check by this code that we only have three subcategory now.

b'Skipping line 6044: expected 10 fields, saw 11\nSkipping line 6569: expected 10 fields, saw 11\nSkipping line 7399: expected 10 fields, saw 11\nSkipping line 7939: expected 10 fields, saw 11\nSkipping line 9026: expected 10 fields, saw 11\nSkipping line 10264: expected 10 fields, saw 11\nSkipping line 10427: expected 10 fields, saw 11\nSkipping line 10905: expected 10 fields, saw 11\nSkipping line 11373: expected 10 fields, saw 11\nSkipping line 11945: expected 10 fields, saw 11\nSkipping line 14112: expected 10 fields, saw 11\nSkipping line 14532: expected 10 fields, saw 11\nSkipping line 15076: expected 10 fields, saw 12\nSkipping line 29906: expected 10 fields, saw 11\nSkipping line 31625: expected 10 fields, saw 11\nSkipping line 33020: expected 10 fields, saw 11\nSkipping line 35748: expected 10 fields, saw 11\nSkipping line 35962: expected 10 fields, saw 11\nSkipping line 37770: expected 10 fields, saw 11\nSkipping line 38105: expected 10 fields, saw 11\nSkipping line 38275: ex

array(['Topwear', 'Bottomwear', 'Footwear'], dtype=object)

In [5]:
"""# Model-1: """

le = LabelEncoder()
#
styles["subCategory"] = le.fit_transform(styles["subCategory"])

styles.head()

#styles["articleType"].unique()


,id,gender,masterCategory,subCategory,articleType,baseColour,season,usage
0,15970,Men,Apparel,2,Shirts,Navy Blue,Fall,Casual
1,39386,Men,Apparel,0,Jeans,Blue,Summer,Casual
3,21379,Men,Apparel,0,Track Pants,Black,Fall,Casual
4,53759,Men,Apparel,2,Tshirts,Grey,Summer,Casual
5,1855,Men,Apparel,2,Tshirts,Grey,Summer,Casual


In [6]:
le.classes_

array(['Bottomwear', 'Footwear', 'Topwear'], dtype=object)

In [7]:
def make_input_array_subcate(df):

    train_images = np.zeros((len(df.id),80,60,3))
    for i in range(len(df.id)):
        
        try:
            ID = df.id.iloc[i]
            path = f"D:/Btech/college/SEM7/archive (1)/images/{ID}.jpg"#/content/images   
            img = cv2.imread(path)
            if img.shape != (80,60,3):#DESIRED SHAPE
                img = image.load_img(path, target_size=(80,60,3))

        except:
            print(ID)
        
        train_images[i] = img

    data = tf.data.Dataset.from_tensor_slices(
      (
        {
          "images" : train_images
       },

        {
          "subCategory" : df[["subCategory"]]
        }
      )
    )

    return data

In [8]:
def make_input_xx(x):
    
    #make_input_array_subcate(styles)
    print("make input")
    x_input = x
    x_input = x_input.shuffle(buffer_size = len(x_input))

    x_train_size = int(0.6*len(x_input))
    x_val_size   = int(0.2*len(x_input))

    x_train = x_input.take(x_train_size).batch(2)
    x_val   = x_input.skip(x_train_size).take(x_val_size).batch(2)
    x_test  = x_input.skip(x_train_size + x_val_size).batch(2)

    return x_train,x_val,x_test


In [9]:
def make_branch(res_input, n_out, act_type, name):
    z = layers.Dense(512, activation="relu")(res_input)
    z = layers.Dense(256, activation='relu')(z)
    z = layers.Dense(128, activation='relu')(z)
    z = layers.Dense(64, activation='relu')(z)
    z = layers.Dense(n_out)(z)
    z = layers.Activation(act_type, name=name)(z)
    return z

In [10]:
def build_model(width, height):
    res50 = keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(80,60,3))
    res50.trainable=False
    inputs = keras.Input(shape=(width,height,3),name = "images")
    x = res50(inputs, training=False)
    x = layers.Conv2D(32, (2, 2), activation='relu')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(1024, activation='relu')(x)
    # -------------------------

    sub_branch = make_branch(x, len(le.classes_), 'softmax', 'subCategory')

    model = keras.Model(inputs=inputs,
                outputs=[sub_branch]
                       )
    return model

In [ ]:
sub_train,sub_val,sub_test = make_input_xx(make_input_array_subcate(styles))


In [ ]:

sub_model = build_model(80, 60)

sub_model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(sub_model)

In [ ]:
sub_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

sub_history = sub_model.fit(sub_train, 
                    epochs=5, 
                    steps_per_epoch = 2000,
                    validation_data = sub_val)

In [ ]:
#sub_model.save("model_sub")

In [ ]:
test_model = tf.keras.models.load_model("model_sub")

test_model.evaluate(sub_test)

In [ ]:
def get_234_df(x):
    styles = pd.read_csv("D:/Btech/college/SEM7/archive (1)/styles.csv", error_bad_lines=False)
    styles = styles.drop(["productDisplayName"],axis = 1)
    styles = styles.drop(["year"],axis = 1)
    styles = styles[(styles.masterCategory=='Apparel')| (styles.masterCategory=='Footwear')]
    styles = styles.drop(styles[styles["subCategory"] == "Innerwear"].index)
    styles = styles.dropna()
    styles = df_drop(styles,"subCategory", ["Apparel Set", "Dress","Loungewear and Nightwear","Saree","Socks"])
    styles["subCategory"] = styles["subCategory"].transform(lambda x: "Footwear" if(x in ["Shoes","Flip Flops","Sandal"]) else x)
    styles = styles.drop(labels=[6695,16194,32309,36381,40000], axis=0)
    styles = styles[styles.subCategory == x]
    return styles


In [ ]:
top_df = get_234_df("Topwear")
bottom_df = get_234_df("Bottomwear")
foot_df = get_234_df("Footwear")

In [ ]:
def my_le(styles):
    articleTypeLB = LabelEncoder()
    genderLB = LabelEncoder()
    baseColourLB = LabelEncoder()
    seasonLB = LabelEncoder()
    usageLB = LabelEncoder()

    styles['articleType'] = articleTypeLB.fit_transform(styles['articleType'])
    styles['gender'] = genderLB.fit_transform(styles['gender'])
    styles['baseColour'] = baseColourLB.fit_transform(styles['baseColour'])
    styles['season'] = seasonLB.fit_transform(styles['season'])
    styles['usage'] = usageLB.fit_transform(styles['usage'])
    return styles,articleTypeLB,genderLB,baseColourLB,seasonLB,usageLB


In [ ]:
top_df,top_art,top_gen,top_base,top_sea,top_usage = my_le(top_df)
bottom_df,bottom_art,bottom_gen,bottom_base,bottom_sea,bottom_usage = my_le(bottom_df)
foot_df,foot_art,foot_gen,foot_base,foot_sea,foot_usage = my_le(foot_df)

foot_usage.classes_


In [ ]:
def build_model(width, height, articleTypeLB,genderLB,baseColourLB,seasonLB,usageLB):
    """
    build the machine learning model. similar to the previous one
    """

    # -------------------------
    res50 = keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(80,60,3))
    res50.trainable=False
    inputs = keras.Input(shape=(width,height,3),name = "images")
    x = res50(inputs, training=False)

    x = layers.Flatten()(x)
    x = layers.Dense(1024, activation='relu')(x)
    # -------------------------

    article_branch = make_branch(x, len(articleTypeLB.classes_), 'softmax', 'articleType')
    gender_branch = make_branch(x, len(genderLB.classes_), 'softmax', 'gender')
    color_branch = make_branch(x, len(baseColourLB.classes_), 'softmax', 'baseColour')
    season_branch = make_branch(x, len(seasonLB.classes_), 'softmax', 'season')
    usage_branch = make_branch(x, len(usageLB.classes_), 'softmax', 'usage')

    model = keras.Model(inputs=inputs,
                outputs=[article_branch, gender_branch, color_branch, 
                            season_branch, usage_branch]
                       )
    return model

In [ ]:
top_base_model = build_model(80,60,top_art,top_gen,top_base,top_sea,top_usage)
bottom_base_model = build_model(80,60,bottom_art,bottom_gen,bottom_base,bottom_sea,bottom_usage)
foot_base_model = build_model(80,60,foot_art,foot_gen,foot_base,foot_sea,foot_usage)


In [ ]:
def make_input_array_2(df):
    train_images = np.zeros((len(df.id),80,60,3))
    for i in range(len(df.id)):
        ID = df.id.iloc[i]
        path = f"D:/Btech/college/SEM7/archive (1)/images/{ID}.jpg"#/content/images   
        img = cv2.imread(path)
        if img.shape != (80,60,3):
            img = image.load_img(path, target_size=(80,60,3))
        
        train_images[i] = img
    
    data = tf.data.Dataset.from_tensor_slices(
      (
        {
          "images" : train_images
       },

        {
          "articleType" : df[["articleType"]],
            'gender' : df[['gender']],
            'baseColour' : df[['baseColour']],
            'season' : df[['season']],
            'usage' : df[['usage']]
            
        }
      )
    )

    return data


In [ ]:
top_train, top_val, top_test = make_input_xx(make_input_array_2(top_df))
bottom_train, bottom_val, bottom_test = make_input_xx(make_input_array_2(bottom_df))
foot_train, foot_val, foot_test = make_input_xx(make_input_array_2(foot_df))


In [ ]:
top_base_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
bottom_base_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
foot_base_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

top_history = top_base_model.fit(top_train, 
                    epochs=10, 
                    steps_per_epoch = 500,
                    validation_data = top_val)

top_base_model.evaluate(top_test)


In [ ]:
#top_base_model.save("model_2.1")


In [ ]:
bottom_history = bottom_base_model.fit(bottom_train, 
                    epochs=15, 
                    steps_per_epoch = 50,
                    validation_data = bottom_val)

bottom_base_model.evaluate(bottom_test)


In [ ]:
#bottom_base_model.save("/content/drive/MyDrive/models/model_2.2")

In [ ]:
foot_history = foot_base_model.fit(foot_train, 
                    epochs=5, 
                    steps_per_epoch = 2000,
                    validation_data = foot_val)

foot_base_model.evaluate(foot_test)

#foot_base_model.save("/content/drive/MyDrive/model_2.2")
